## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint George,US,37.1041,-113.5841,32.79,59,1,6.91,clear sky
1,1,Hobyo,SO,5.3505,48.5268,79.36,84,55,14.18,broken clouds
2,2,Kapaa,US,22.0752,-159.3190,71.19,84,75,8.05,moderate rain
3,3,Naqadeh,IR,36.9553,45.3880,26.87,77,19,4.32,few clouds
4,4,Mataura,NZ,-46.1927,168.8643,60.48,75,100,0.60,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hobyo,SO,5.3505,48.5268,79.36,84,55,14.18,broken clouds
8,8,Kimbe,PG,-5.5502,150.1429,76.24,94,100,2.13,moderate rain
17,17,Avarua,CK,-21.2078,-159.7750,77.05,88,20,6.91,few clouds
18,18,Mitsamiouli,KM,-11.3847,43.2844,88.09,66,20,9.22,few clouds
23,23,Puerto Ayora,EC,-0.7393,-90.3518,76.96,77,71,7.76,broken clouds
27,27,Saint-Georges,GD,12.0564,-61.7485,83.88,70,75,18.41,broken clouds
32,32,Rikitea,PF,-23.1203,-134.9692,77.22,67,87,3.67,overcast clouds
34,34,Faanui,PF,-16.4833,-151.7500,78.80,78,100,8.97,light rain
38,38,Omboue,GA,-1.5746,9.2618,80.24,79,78,7.94,broken clouds
45,45,Atuona,PF,-9.8000,-139.0333,78.13,75,24,17.92,light rain


In [7]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()


City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hobyo,SO,5.3505,48.5268,79.36,84,55,14.18,broken clouds
8,8,Kimbe,PG,-5.5502,150.1429,76.24,94,100,2.13,moderate rain
17,17,Avarua,CK,-21.2078,-159.7750,77.05,88,20,6.91,few clouds
18,18,Mitsamiouli,KM,-11.3847,43.2844,88.09,66,20,9.22,few clouds
23,23,Puerto Ayora,EC,-0.7393,-90.3518,76.96,77,71,7.76,broken clouds
...,...,...,...,...,...,...,...,...,...,...
663,663,Oistins,BB,13.0667,-59.5333,84.29,54,20,24.16,few clouds
664,664,Kudahuvadhoo,MV,2.6708,72.8944,81.39,78,81,9.80,broken clouds
666,666,Impfondo,CG,1.6381,18.0667,86.52,36,100,2.17,overcast clouds
677,677,Makasar,ID,-5.1400,119.4221,77.05,100,40,5.75,scattered clouds


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobyo,SO,79.36,broken clouds,5.3505,48.5268,
8,Kimbe,PG,76.24,moderate rain,-5.5502,150.1429,
17,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,
18,Mitsamiouli,KM,88.09,few clouds,-11.3847,43.2844,
23,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
27,Saint-Georges,GD,83.88,broken clouds,12.0564,-61.7485,
32,Rikitea,PF,77.22,overcast clouds,-23.1203,-134.9692,
34,Faanui,PF,78.80,light rain,-16.4833,-151.7500,
38,Omboue,GA,80.24,broken clouds,-1.5746,9.2618,
45,Atuona,PF,78.13,light rain,-9.8000,-139.0333,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
         print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobyo,SO,79.36,broken clouds,5.3505,48.5268,Halane Hotel and Restaurent
8,Kimbe,PG,76.24,moderate rain,-5.5502,150.1429,Kimbe Bay Hotel
17,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,Paradise Inn
18,Mitsamiouli,KM,88.09,few clouds,-11.3847,43.2844,Foyer ADM
23,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
663,Oistins,BB,84.29,few clouds,13.0667,-59.5333,Butterfly Beach Hotel
664,Kudahuvadhoo,MV,81.39,broken clouds,2.6708,72.8944,Niyama Private Islands Maldives
666,Impfondo,CG,86.52,overcast clouds,1.6381,18.0667,Auberge La Place de Jules
677,Makasar,ID,77.05,scattered clouds,-5.1400,119.4221,Hotel Horison Ultima Makassar


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))